Access token: NLSQjUEWly7uv5AK1-fwSwXXl0QN9eQSYlJ2DP_0qpCepn1MLaXozB6VSvkqz1qX

ERROR: Could not find a version that satisfies the requirement glove-python-binary (from versions: none)
ERROR: No matching distribution found for glove-python-binary


In [1]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import numpy as np

In [2]:
def get_artist_id(artist_name, access_token):
    url = "https://api.genius.com/search"
    headers = {"Authorization": f"Bearer {access_token}"}
    params = {"q": artist_name}
    
    response = requests.get(url, params=params, headers=headers).json()
    hits = response["response"]["hits"]
    
    for hit in hits:
        if hit["result"]["primary_artist"]["name"].lower() == artist_name.lower():
            return hit["result"]["primary_artist"]["id"]
    return None

def get_artist_songs(artist_name, artist_id, access_token, max_songs=10):
    songs = []
    page = 1
    headers = {"Authorization": f"Bearer {access_token}"}
    
    while len(songs) < max_songs:
        url = f"https://api.genius.com/artists/{artist_id}/songs"
        params = {"per_page": 20, "page": page, "sort": "popularity"}
        response = requests.get(url, params=params, headers=headers).json()
        new_songs = response["response"]["songs"]
        
        if not new_songs:
            break
        
        for song in new_songs:
            if song["primary_artist"]["id"] == artist_id:
                songs.append({
                    "title": song["title"],
                    "url": song["url"],
                    "release_date": song.get("release_date", "N/A"),
                    "artist": artist_name
                })
                if len(songs) >= max_songs:
                    break
        page += 1
        
    return songs

def get_lyrics_from_url(song_url):
    page = requests.get(song_url)
    soup = BeautifulSoup(page.text, 'html.parser')
    lyrics = []

    # Find the div that contains the lyrics
    lyrics_div = soup.find("div", attrs={"data-lyrics-container": "true"})
    
    # Use .decode_contents() to get the raw HTML (including <br> tags)
    lyric_links = lyrics_div.find_all("a", attrs={"data-ignore-on-click-outside": "true"})
    
    for link in lyric_links:
        # Get the raw HTML content
        lyric_html = link.find("span").decode_contents()

        # Replace <br> tags with a whitespace
        lyric_text = re.sub(r"<br\s*/?>", " ", lyric_html)
        
        # Remove bracketed sections (like [Chorus], [Verse 1], etc.)
        lyric_text = re.sub(r"\[.*?\]", "", lyric_text)
        
        # Remove parentheses (if needed)
        lyric_text = re.sub(r"[()]", "", lyric_text)
        
        lyrics.append(lyric_text)

    # Split the lyrics into words, removing punctuation, and keeping apostrophes
    words = [re.sub(r"[^\w']", '', word) for sentence in lyrics for word in sentence.split()]
    words = [word for word in words if word]  # remove empty strings

    return words
    


def get_kendrick_lyrics(access_token, max_songs=5):
    artist_id = get_artist_id("Kendrick Lamar", access_token)
    if not artist_id:
        return "Artist not found."

    songs = get_artist_songs("Kendrick Lamar", artist_id, access_token, max_songs)
    for song in songs:
        print(f"Fetching lyrics for: {song['title']}")
        lyrics = get_lyrics_from_url(song["url"])
        song["lyrics"] = lyrics
    return songs

In [3]:
# access_token = "NLSQjUEWly7uv5AK1-fwSwXXl0QN9eQSYlJ2DP_0qpCepn1MLaXozB6VSvkqz1qX"
# kendrick_songs = get_kendrick_lyrics(access_token, max_songs=10)

# for song in kendrick_songs:
#     print(f"\n{song['title']} ({song['release_date']})\n")
#     print(song['lyrics'][:500])  # preview first 500 chars
#     print("...\n" + "-"*80)
#     print(song["url"])


In [4]:
def get_artist_lyrics(access_token, artist, max_songs=5):
    artist_id = get_artist_id(artist, access_token)
    if not artist_id:
        return "Artist not found."

    songs = get_artist_songs(artist, artist_id, access_token, max_songs)
    for song in songs:
        print(f"Fetching lyrics for: {song['title']}")
        lyrics = get_lyrics_from_url(song["url"])
        song["lyrics"] = lyrics
    return songs

In [ ]:
pop_culture_artists = ["Kendrick Lamar", "Drake"]

In [ ]:

song_list = []

access_token = "NLSQjUEWly7uv5AK1-fwSwXXl0QN9eQSYlJ2DP_0qpCepn1MLaXozB6VSvkqz1qX"
all_songs = []
for artist in pop_culture_artists:
    all_songs.append(get_artist_lyrics(access_token, artist, max_songs=50))

for thing in all_songs:
    for song in thing:
        song_list.append([song["title"], song["artist"], song["lyrics"]])
    

song_info = pd.DataFrame(song_list)
song_info

Fetching lyrics for: The Hills
Fetching lyrics for: Starboy
Fetching lyrics for: Die For You
Fetching lyrics for: Can’t Feel My Face
Fetching lyrics for: Reminder
Fetching lyrics for: lovely
Fetching lyrics for: when the party’s over
Fetching lyrics for: Happier Than Ever
Fetching lyrics for: bad guy
Fetching lyrics for: everything i wanted
Fetching lyrics for: Not Like Us
Fetching lyrics for: HUMBLE.
Fetching lyrics for: euphoria
Fetching lyrics for: m.A.A.d city
Fetching lyrics for: Swimming Pools (Drank)
Fetching lyrics for: God’s Plan
Fetching lyrics for: In My Feelings
Fetching lyrics for: Hotline Bling
Fetching lyrics for: One Dance
Fetching lyrics for: Hold On, We’re Going Home


,0,1,2
0,The Hills,The Weeknd,"[Your, man, on, the, road, he, doin', promo, Y..."
1,Starboy,The Weeknd,"[I'm, tryna, put, you, in, the, worst, mood, a..."
2,Die For You,The Weeknd,"[I'm, findin', ways, to, articulate, the, feel..."
3,Can’t Feel My Face,The Weeknd,"[And, I, know, she'll, be, the, death, of, me,..."
4,Reminder,The Weeknd,"[Record, man, play, my, song, on, the, radio, ..."
5,lovely,Billie Eilish,"[Thought, I, found, a, way, Thought, I, found,..."
6,when the party’s over,Billie Eilish,"[Tore, my, shirt, to, stop, you, bleedin', Qui..."
7,Happier Than Ever,Billie Eilish,"[Wish, I, could, explain, it, better, I, wish,..."
8,bad guy,Billie Eilish,"[White, shirt, now, red, my, bloody, nose, Bru..."
9,everything i wanted,Billie Eilish,"[I, had, a, dream, I, got, everything, I, want..."


In [12]:
!pip install keras

In [21]:
song_info.columns = ["Song", "Artist", "Lyrics"]

In [22]:
from itertools import chain

def get_artist_corpus(df, artist_name):
    artist_lyrics = df[df["Artist"] == artist_name]["Lyrics"].tolist()
    return list(chain.from_iterable(artist_lyrics))

get_artist_corpus(song_info, "Drake")

['And',
 'they',
 "wishin'",
 'and',
 "wishin'",
 'And',
 "wishin'",
 'and',
 "wishin'",
 'they',
 "wishin'",
 'on',
 'me',
 'Yeah',
 'I',
 'been',
 "movin'",
 'calm',
 "don't",
 'start',
 'no',
 'trouble',
 'with',
 'me',
 'Tryna',
 'keep',
 'it',
 'peaceful',
 'is',
 'a',
 'struggle',
 'for',
 'me',
 "Don't",
 'pull',
 'up',
 'at',
 '6',
 'am',
 'to',
 'cuddle',
 'with',
 'me',
 'You',
 'know',
 'how',
 'I',
 'like',
 'it',
 'when',
 'you',
 "lovin'",
 'on',
 'me',
 'I',
 "don't",
 'wanna',
 'die',
 'for',
 'them',
 'to',
 'miss',
 'me',
 'Yes',
 'I',
 'see',
 'the',
 'things',
 'that',
 'they',
 "wishin'",
 'on',
 'me',
 'Hope',
 'I',
 'got',
 'some',
 'brothers',
 'that',
 'outlive',
 'me',
 'They',
 "gon'",
 'tell',
 'the',
 'story',
 'shit',
 'was',
 'different',
 'with',
 'me',
 "God's",
 'plan',
 "God's",
 'plan',
 'I',
 'hold',
 'back',
 'sometimes',
 'I',
 "won't",
 'yeah',
 'I',
 'feel',
 'good',
 'sometimes',
 'I',
 "don't",
 'Ayy',
 "don't",
 'I',
 'finessed',
 'down',
 'W

In [14]:
from itertools import chain
import nltk
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences


In [15]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\1khal\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [16]:
def prepare_sequences(corpus, sequence_length=5):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts([" ".join(corpus)])

    total_words = len(tokenizer.word_index) + 1

    # Create input sequences
    input_sequences = []
    for i in range(sequence_length, len(corpus)):
        seq = corpus[i-sequence_length:i+1]
        encoded = tokenizer.texts_to_sequences([" ".join(seq)])[0]
        input_sequences.append(encoded)

    input_sequences = np.array(input_sequences)
    X, y = input_sequences[:, :-1], input_sequences[:, -1]
    y = to_categorical(y, num_classes=total_words)

    return X, y, tokenizer, total_words

In [17]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

def build_model(input_len, total_words):
    model = Sequential()
    model.add(Embedding(total_words, 64, input_length=input_len))
    model.add(LSTM(100))
    model.add(Dense(total_words, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [24]:
corpus = get_artist_corpus(song_info, "Drake")
X, y, tokenizer, total_words = prepare_sequences(corpus)
model = build_model(X.shape[1], total_words)
model.fit(X, y, epochs=100, verbose=1)

Epoch 1/100


c:\Users\1khal\miniforge3\envs\dsc80\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


22/22 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - accuracy: 0.0422 - loss: 5.3361  
Epoch 2/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.0851 - loss: 5.2460
Epoch 3/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.0711 - loss: 4.7972
Epoch 4/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.0619 - loss: 4.7006 
Epoch 5/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.1014 - loss: 4.5415
Epoch 6/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.0852 - loss: 4.4852
Epoch 7/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.0873 - loss: 4.3786
Epoch 8/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.0993 - loss: 4.3151
Epoch 9/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.1072 - loss: 4.2749
Epoch 10/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.1478 - loss: 4.0684
Epoch 11/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.1396 - loss: 4.0461
Epoch 12/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.1696 -

In [25]:
def generate_lyrics(seed_text, tokenizer, model, length=20):
    result = seed_text.split()
    for _ in range(length):
        encoded = tokenizer.texts_to_sequences([" ".join(result[-5:])])[0]
        encoded = pad_sequences([encoded], maxlen=5, truncating='pre')
        predicted = model.predict(encoded, verbose=0)
        next_word = tokenizer.index_word[np.argmax(predicted)]
        result.append(next_word)
    return " ".join(result)

In [26]:
generate_lyrics("love story baby", tokenizer, model, length=15)


"love story baby a i pray i your there's why i they good knows on and things really"